In [3]:
import planetary_computer
planetary_computer.settings.set_subscription_key("sofiasarak")

In [7]:
import pandas as pd
import planetary_computer
import pystac_client
# import rich.table
# import stackstac
# from geogif import gif

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
bbox = [-112.826843, 32.974108, -111.184387, 33.863574]
search = catalog.search(collections=["io-biodiversity"], bbox=bbox_of_interest)

items = list(search.items())
for item in items:
    print(item)

NameError: name 'bbox_of_interest' is not defined

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rioxr

from pystac_client import Client  # To access STAC catalogs

import planetary_computer  # To sign items from the MPC STAC catalog 

from IPython.display import Image  # To nicely display images

In [3]:
# Access MPC catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [4]:
bii = catalog.get_child('io-biodiversity')

In [7]:
range_2017 = "2017-01-01/2020-12-31"
range_2020 = "2020-01-01/2020-12-31"


bbox = {
    "type": "Polygon",
    "coordinates":[
        [
            [-112.826843, 32.974108, -111.184387, 33.863574]
        ]
    ],
}

In [8]:
search_2017 = catalog.search(
    collections = ['io-biodiversity'],
    intersects = bbox,
    datetime = range_2017)
search_2017

In [9]:
# Retrieve search items
items = search_2017.item_collection()
len(items)

APIError: [{'type': 'literal_error', 'loc': ('body', 'intersects', 'Point', 'type'), 'msg': "Input should be 'Point'", 'input': 'Polygon', 'ctx': {'expected': "'Point'"}}, {'type': 'float_type', 'loc': ('body', 'intersects', 'Point', 'coordinates', 'call[Position2D]', 0), 'msg': 'Input should be a valid number', 'input': [[-112.826843, 32.974108, -111.184387, 33.863574]]}, {'type': 'missing_argument', 'loc': ('body', 'intersects', 'Point', 'coordinates', 'call[Position2D]', 'latitude'), 'msg': 'Missing required argument', 'input': [[[-112.826843, 32.974108, -111.184387, 33.863574]]]}, {'type': 'float_type', 'loc': ('body', 'intersects', 'Point', 'coordinates', 'call[Position3D]', 0), 'msg': 'Input should be a valid number', 'input': [[-112.826843, 32.974108, -111.184387, 33.863574]]}, {'type': 'missing_argument', 'loc': ('body', 'intersects', 'Point', 'coordinates', 'call[Position3D]', 'latitude'), 'msg': 'Missing required argument', 'input': [[[-112.826843, 32.974108, -111.184387, 33.